In [1]:
import tensorflow as tf

#checking GPU's condition
tf.test.is_built_with_cuda()
print(tf.__version__)
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
tf.config.list_physical_devices('GPU')
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.3.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1082828942407910041
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15481167098233523246
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12236342270375296863
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3455057920
locality {
  bus_id: 1
  links {
  }
}
incarnation: 651028302537324578
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras
import imblearn

# Uploading Data

In [3]:
#Splitting data into X & Y.

df = pd.read_csv("croppred.csv")
data = pd.DataFrame(df)
# data = data.drop(columns = ["rainfall"])
# data = data.drop(columns = ["temp"])
data = data.dropna()


    
pred = data[["crop"]]
data = data.drop(columns = ["crop"])
data = data.astype(str)
pred = pred.astype(str)
data = pd.get_dummies(data,prefix='',prefix_sep ='')
#pred.info()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
pred = le.fit_transform(pred)

# pd.concat([pred,om])
# pred = pred.drop(columns = ["crop"]
# pred = pd.get_dummies(pred,prefix='',prefix_sep ='')
# pred = pred.to_numpy()
np.unique(pred)
    
            

/home/naman/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [5]:
len(data)

5037

In [13]:
from tensorflow import keras
pred = keras.utils.to_categorical(pred,11)
pred.shape

(10120, 11, 11)

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
# pred = sc.fit_transform(pred)
data = sc.fit_transform(data)

len(data)

5037

In [8]:
# from scipy.sparse import coo_matrix
# xtrainspar = coo_matrix(data)
# from sklearn.utils import resample
# data, xtrainspar, pred = resample(data, xtrainspar,pred, random_state=0)
from imblearn.over_sampling import RandomOverSampler
rus = RandomOverSampler(random_state=11)
data,pred = rus.fit_resample(data,pred)
len(data)

/home/naman/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


10120

In [9]:
# Splitting data into test, train & validation

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data, pred, test_size = 0.2)

In [10]:
# 80% data for train, 10% for test & 10% for validation



# Model 

In [11]:
from tensorflow.keras.regularizers import l2
l2_reg = 0.001
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units= 116,activation= 'relu', input_shape = (116,),kernel_regularizer = l2(l2_reg)))
model.add(tf.keras.layers.Dense(units= 600,activation= 'relu',kernel_regularizer = l2(l2_reg)))

model.add(tf.keras.layers.Dense(units= 400,activation= 'relu',kernel_regularizer = l2(l2_reg)))
model.add(tf.keras.layers.Dense(units= 200,activation= 'relu',kernel_regularizer = l2(l2_reg)
))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units= 20, activation= 'relu'))




model.add(tf.keras.layers.Dense(units = 11, activation= 'softmax'))
opt = keras.optimizers.Adam(lr = 0.001)
mse = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 116)               13572     
_________________________________________________________________
dense_1 (Dense)              (None, 600)               70200     
_________________________________________________________________
dense_2 (Dense)              (None, 400)               240400    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                4020      
_________________________________________________________________
dense_5 (Dense)              (None, 11)                2

In [12]:
model.fit(xtrain,ytrain, epochs=500,validation_split=0.2)

Epoch 1/500
203/203 [==============================] - 1s 6ms/step - loss: 2.4037 - accuracy: 0.1997 - val_loss: 2.0076 - val_accuracy: 0.2506
Epoch 2/500
203/203 [==============================] - 1s 5ms/step - loss: 1.9580 - accuracy: 0.2444 - val_loss: 1.9070 - val_accuracy: 0.2654
Epoch 3/500
203/203 [==============================] - 1s 5ms/step - loss: 1.8711 - accuracy: 0.2699 - val_loss: 1.8222 - val_accuracy: 0.2938
Epoch 4/500
203/203 [==============================] - 1s 5ms/step - loss: 1.8189 - accuracy: 0.3108 - val_loss: 1.7887 - val_accuracy: 0.3383
Epoch 5/500
203/203 [==============================] - 1s 5ms/step - loss: 1.7830 - accuracy: 0.3323 - val_loss: 1.7889 - val_accuracy: 0.3148
Epoch 6/500
203/203 [==============================] - 1s 5ms/step - loss: 1.7575 - accuracy: 0.3450 - val_loss: 1.7499 - val_accuracy: 0.3556
Epoch 7/500
203/203 [==============================] - 1s 5ms/step - loss: 1.7294 - accuracy: 0.3647 - val_loss: 1.7423 - val_accuracy: 0.3753

Epoch 58/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3489 - accuracy: 0.5216 - val_loss: 1.5547 - val_accuracy: 0.4753
Epoch 59/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3616 - accuracy: 0.5199 - val_loss: 1.5826 - val_accuracy: 0.4444
Epoch 60/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3535 - accuracy: 0.5250 - val_loss: 1.5460 - val_accuracy: 0.4648
Epoch 61/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3544 - accuracy: 0.5250 - val_loss: 1.5541 - val_accuracy: 0.4728
Epoch 62/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3525 - accuracy: 0.5178 - val_loss: 1.5512 - val_accuracy: 0.4630
Epoch 63/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3488 - accuracy: 0.5153 - val_loss: 1.5332 - val_accuracy: 0.4728
Epoch 64/500
203/203 [==============================] - 1s 5ms/step - loss: 1.3481 - accuracy: 0.5181 - val_loss: 1.5544 - val_accuracy:

Epoch 115/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2837 - accuracy: 0.5381 - val_loss: 1.6044 - val_accuracy: 0.4543
Epoch 116/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2670 - accuracy: 0.5545 - val_loss: 1.5913 - val_accuracy: 0.4562
Epoch 117/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2676 - accuracy: 0.5483 - val_loss: 1.5748 - val_accuracy: 0.4630
Epoch 118/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2655 - accuracy: 0.5422 - val_loss: 1.5902 - val_accuracy: 0.4623
Epoch 119/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2659 - accuracy: 0.5463 - val_loss: 1.5805 - val_accuracy: 0.4586
Epoch 120/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2658 - accuracy: 0.5485 - val_loss: 1.5861 - val_accuracy: 0.4549
Epoch 121/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2649 - accuracy: 0.5406 - val_loss: 1.5878 - val_ac

203/203 [==============================] - 1s 5ms/step - loss: 1.2289 - accuracy: 0.5599 - val_loss: 1.5727 - val_accuracy: 0.4685
Epoch 172/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2241 - accuracy: 0.5556 - val_loss: 1.5753 - val_accuracy: 0.4716
Epoch 173/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2314 - accuracy: 0.5488 - val_loss: 1.5737 - val_accuracy: 0.4654
Epoch 174/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2336 - accuracy: 0.5530 - val_loss: 1.5971 - val_accuracy: 0.4716
Epoch 175/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2268 - accuracy: 0.5619 - val_loss: 1.5896 - val_accuracy: 0.4636
Epoch 176/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2191 - accuracy: 0.5590 - val_loss: 1.6249 - val_accuracy: 0.4679
Epoch 177/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2243 - accuracy: 0.5534 - val_loss: 1.5976 - val_accuracy: 0.4648

203/203 [==============================] - 1s 5ms/step - loss: 1.2025 - accuracy: 0.5544 - val_loss: 1.7000 - val_accuracy: 0.4605
Epoch 228/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2074 - accuracy: 0.5636 - val_loss: 1.6476 - val_accuracy: 0.4710
Epoch 229/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1996 - accuracy: 0.5622 - val_loss: 1.6286 - val_accuracy: 0.4716
Epoch 230/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2068 - accuracy: 0.5584 - val_loss: 1.5963 - val_accuracy: 0.4673
Epoch 231/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2022 - accuracy: 0.5601 - val_loss: 1.6146 - val_accuracy: 0.4642
Epoch 232/500
203/203 [==============================] - 1s 5ms/step - loss: 1.2059 - accuracy: 0.5599 - val_loss: 1.6308 - val_accuracy: 0.4691
Epoch 233/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1992 - accuracy: 0.5607 - val_loss: 1.6538 - val_accuracy: 0.4691

203/203 [==============================] - 1s 5ms/step - loss: 1.1878 - accuracy: 0.5553 - val_loss: 1.6368 - val_accuracy: 0.4605
Epoch 284/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1884 - accuracy: 0.5618 - val_loss: 1.6937 - val_accuracy: 0.4481
Epoch 285/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1856 - accuracy: 0.5633 - val_loss: 1.6538 - val_accuracy: 0.4796
Epoch 286/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1831 - accuracy: 0.5639 - val_loss: 1.6441 - val_accuracy: 0.4778
Epoch 287/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1878 - accuracy: 0.5638 - val_loss: 1.6220 - val_accuracy: 0.4716
Epoch 288/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1903 - accuracy: 0.5561 - val_loss: 1.6309 - val_accuracy: 0.4728
Epoch 289/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1985 - accuracy: 0.5619 - val_loss: 1.6485 - val_accuracy: 0.4704

203/203 [==============================] - 1s 5ms/step - loss: 1.1759 - accuracy: 0.5564 - val_loss: 1.7085 - val_accuracy: 0.4747
Epoch 340/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1751 - accuracy: 0.5632 - val_loss: 1.6664 - val_accuracy: 0.4716
Epoch 341/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1785 - accuracy: 0.5630 - val_loss: 1.6617 - val_accuracy: 0.4704
Epoch 342/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1825 - accuracy: 0.5604 - val_loss: 1.6319 - val_accuracy: 0.4833
Epoch 343/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1808 - accuracy: 0.5587 - val_loss: 1.6548 - val_accuracy: 0.4580
Epoch 344/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1833 - accuracy: 0.5602 - val_loss: 1.6409 - val_accuracy: 0.4821
Epoch 345/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1820 - accuracy: 0.5633 - val_loss: 1.6438 - val_accuracy: 0.4728

203/203 [==============================] - 1s 5ms/step - loss: 1.1713 - accuracy: 0.5645 - val_loss: 1.6551 - val_accuracy: 0.4827
Epoch 396/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1679 - accuracy: 0.5684 - val_loss: 1.6711 - val_accuracy: 0.4759
Epoch 397/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1667 - accuracy: 0.5615 - val_loss: 1.6598 - val_accuracy: 0.4673
Epoch 398/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1652 - accuracy: 0.5661 - val_loss: 1.6687 - val_accuracy: 0.4735
Epoch 399/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1656 - accuracy: 0.5598 - val_loss: 1.6855 - val_accuracy: 0.4722
Epoch 400/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1649 - accuracy: 0.5726 - val_loss: 1.6671 - val_accuracy: 0.4802
Epoch 401/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1659 - accuracy: 0.5645 - val_loss: 1.6278 - val_accuracy: 0.4815

203/203 [==============================] - 1s 5ms/step - loss: 1.1584 - accuracy: 0.5650 - val_loss: 1.6720 - val_accuracy: 0.4728
Epoch 452/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1539 - accuracy: 0.5633 - val_loss: 1.7623 - val_accuracy: 0.4648
Epoch 453/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1605 - accuracy: 0.5645 - val_loss: 1.6506 - val_accuracy: 0.4716
Epoch 454/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1596 - accuracy: 0.5611 - val_loss: 1.7145 - val_accuracy: 0.4722
Epoch 455/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1564 - accuracy: 0.5659 - val_loss: 1.6998 - val_accuracy: 0.4691
Epoch 456/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1573 - accuracy: 0.5652 - val_loss: 1.6887 - val_accuracy: 0.4648
Epoch 457/500
203/203 [==============================] - 1s 5ms/step - loss: 1.1551 - accuracy: 0.5638 - val_loss: 1.7376 - val_accuracy: 0.4790

In [14]:
nb = model.predict(xtest)

In [15]:
pre = np.argsort(nb[0])
pre

array([ 2,  5,  3,  6,  7, 10,  8,  4,  9,  1,  0])

In [16]:
np.argsort(ytest[0])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0])

In [17]:
np.sum(nb[0])

1.0

In [37]:
# serialize to JSON
json_file = model.to_json()

with open('model.json', "w") as file:
     file.write(json_file)


# serialize weights to HDF5
model.save_weights('model.h5')


In [38]:
from tensorflow.keras.models import model_from_json
# load json and create model
file = open('model.json', 'r')
model_json = file.read()
file.close()
lmodel = model_from_json(model_json)
# load weights
lmodel.load_weights('model.h5')
aa =lmodel.predict(xtest)
pr = np.argsort(nb[0])
pr

array([ 2,  5,  3,  6,  7, 10,  8,  4,  9,  1,  0])